In [43]:
import networkx as nx
import ndlib.models.ModelConfig as mc
import ndlib.models.epidemics as ep
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
from ndlib.viz.mpl.DiffusionTrend import DiffusionTrend


In [ ]:

# Create a graph (e.g., Erdős-Rényi graph)
# G = nx.erdos_renyi_graph(n=100, p=0.1)

G = nx.read_edgelist('./../datasets/BA_EXP/ba_edgelist_exp3_4000_10.edges', comments="%", nodetype=int)

In [45]:
G.number_of_nodes()

4000

In [46]:


# Get the mean degree (k) of the graph
degrees = [deg for _, deg in G.degree()]
mean_degree = np.mean(degrees)

# Calculate B_Threshold
B_Threshold = mean_degree / (mean_degree**2 - mean_degree)

# Range of B values
B_values = np.linspace(1 * B_Threshold, 1.9 * B_Threshold, 5)

# Number of iterations (M)
num_iterations = 100
num_nodes = G.number_of_nodes()

affected_scales = {}


In [47]:
B_values

array([0.11126565, 0.13630042, 0.16133519, 0.18636996, 0.21140473])

In [48]:
print(mean_degree)

# Second moment (mean of squared degrees)
mean_degree_squared = np.mean([deg**2 for deg in degrees])
print(mean_degree_squared)

# Epidemic threshold (B_Threshold)
B_Threshold = mean_degree / (mean_degree_squared - mean_degree)
# Range of B values
B_values = np.linspace(1 * B_Threshold, 2 * B_Threshold, 5)
B_values

9.9875
250.301


array([0.0415603 , 0.05195037, 0.06234044, 0.07273052, 0.08312059])

In [49]:

infected_scales = {}
for B in B_values:
    recovered_sum = 0  # To store the sum of recovered nodes across all iterations
    infected_sum = 0
    # Store trends for plotting
    trends = []

    for i in range(num_iterations):
        # Initialize the SIR model
        model = ep.SIRModel(G)
        
        # Configuration setup
        config = mc.Configuration()
        config.add_model_parameter('beta', B)  # Set infection rate to current B
        config.add_model_parameter('gamma', 1.0)  # Recovery probability = 1
        config.add_model_initial_configuration("Infected", {0: 1})  # Start with node 0 infected
        
        # Set the model configuration
        model.set_initial_status(config)
        
        iteration = None
        # Run the model until all nodes are either recovered or susceptible
        for step in range(200):  # Maximum 200 steps
            iteration = model.iteration()
            trends.append(model.build_trends([iteration]))
            
            # Check if all nodes are either recovered or susceptible (no infected nodes left)
            if iteration['node_count'][1] == 0:  # Index 1 corresponds to 'Infected'
                break  # Exit the loop if no infected nodes remain
        
        # Get the final state after the infection spread
        final_state = iteration['node_count']
        if i%10==0:
            print(final_state)
        recovered_nodes = final_state[2]  # Index 2 represents 'Recovered' nodes
        recovered_sum += recovered_nodes
        infected_sum += final_state[1]

    
    # Calculate the affected scale for the current B
    affected_scale = recovered_sum / (num_iterations * num_nodes)
    affected_scales[round(B, 3)] = round(affected_scale, 3)
    infected_scales[round(B, 3)] = infected_sum 
    
    susceptible_values = []
    infected_values = []
    recovered_values = []
    
    for trend_list in trends:  # Iterating over the list of trend data
        for trend_dict in trend_list:  # Extracting the trend dictionaries
            if 'trends' in trend_dict:  # Ensure the dictionary contains 'trends'
                node_count = trend_dict['trends']['node_count']
                if len(node_count) >= 3:
                    susceptible_values.append(node_count[0][0])  # Susceptible
                    infected_values.append(node_count[1][0])  # Infected
                    recovered_values.append(node_count[2][0])  # Recovered

    plt.figure()
    plt.scatter(range(len(susceptible_values)), susceptible_values, label='Susceptible', marker='o')
    plt.scatter(range(len(recovered_values)), recovered_values, label='Recovered', marker='^')
    plt.scatter(range(len(infected_values)), infected_values, label='Infected', marker='x')
    plt.title(f"Diffusion Trend for B={round(B, 3)} - Iteration {i+1}")
    plt.xlabel("Time Step")
    plt.ylabel("Number of Nodes")
    plt.legend()
    plt.savefig(f"plot_for_B={round(B, 3)}.png", dpi=300)
    plt.close()


{0: 3921, 1: 0, 2: 79}
{0: 3980, 1: 0, 2: 20}
{0: 3996, 1: 0, 2: 4}
{0: 3953, 1: 0, 2: 47}
{0: 3979, 1: 0, 2: 21}
{0: 3970, 1: 0, 2: 30}
{0: 3993, 1: 0, 2: 7}
{0: 3969, 1: 0, 2: 31}
{0: 3909, 1: 0, 2: 91}
{0: 3976, 1: 0, 2: 24}
{0: 3972, 1: 0, 2: 28}
{0: 3753, 1: 0, 2: 247}
{0: 3977, 1: 0, 2: 23}
{0: 3995, 1: 0, 2: 5}
{0: 3978, 1: 0, 2: 22}
{0: 3716, 1: 0, 2: 284}
{0: 3873, 1: 0, 2: 127}
{0: 3928, 1: 0, 2: 72}
{0: 3978, 1: 0, 2: 22}
{0: 3711, 1: 0, 2: 289}
{0: 3594, 1: 0, 2: 406}
{0: 3561, 1: 0, 2: 439}
{0: 3870, 1: 0, 2: 130}
{0: 3834, 1: 0, 2: 166}
{0: 3566, 1: 0, 2: 434}
{0: 3568, 1: 0, 2: 432}
{0: 3587, 1: 0, 2: 413}
{0: 3967, 1: 0, 2: 33}
{0: 3518, 1: 0, 2: 482}
{0: 3577, 1: 0, 2: 423}
{0: 3388, 1: 0, 2: 612}
{0: 3269, 1: 0, 2: 731}
{0: 3295, 1: 0, 2: 705}
{0: 3213, 1: 0, 2: 787}
{0: 3490, 1: 0, 2: 510}
{0: 3343, 1: 0, 2: 657}
{0: 3334, 1: 0, 2: 666}
{0: 3334, 1: 0, 2: 666}
{0: 3398, 1: 0, 2: 602}
{0: 3305, 1: 0, 2: 695}
{0: 3067, 1: 0, 2: 933}
{0: 3077, 1: 0, 2: 923}
{0: 3040, 1:

In [51]:

# Output the affected scales for each B
for B, scale in affected_scales.items():
    print(f"B = {B}: Affected scale = {scale}, infected_avg = {infected_scales[B]}")
print(affected_scales)
print(infected_scales)

B = 0.042: Affected scale = 0.01, infected_avg = 0
B = 0.052: Affected scale = 0.027, infected_avg = 0
B = 0.062: Affected scale = 0.08, infected_avg = 0
B = 0.073: Affected scale = 0.158, infected_avg = 0
B = 0.083: Affected scale = 0.224, infected_avg = 0
{np.float64(0.042): 0.01, np.float64(0.052): 0.027, np.float64(0.062): 0.08, np.float64(0.073): 0.158, np.float64(0.083): 0.224}
{np.float64(0.042): 0, np.float64(0.052): 0, np.float64(0.062): 0, np.float64(0.073): 0, np.float64(0.083): 0}
